In [ ]:
import os
import sys
import hashlib

!pip install tensorflow_addons
!pip install rdkit
!pip install keras-swa

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
import keras.backend as K
from tensorflow.keras import layers
from swa.tfkeras import SWA

from custom_loss import rwrmse, pseudo_huber, alpha_1point75, alpha_1point5, alpha_1point25, alpha_1point125, alpha_point5, alpha_adaptive, cauchy
from custom_loss import rw_cosine_dissimilarity

import math as m
import numpy as np
import pandas as pd
import itertools
import warnings

from rdkit import Chem
from rdkit import RDLogger
from rdkit.Chem import AllChem
from rdkit.Chem import RDKFingerprint

from sklearn.preprocessing import StandardScaler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for keras-swa: filename=keras_swa-0.1.7-py3-none-any.whl size=7823 sha256=cea469816ccee168a42557c83bff6c6c41e4cf8464f21b55786d8ebbe5c61a38
  Stored in directory: /root/.cache/pip/wheels/1f/f3/68/48c5eb0509cd523f1c975e1240ae3d97540f6e2666bcdedae4
Successfully built keras-swa


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
# Set random seeds
np.random.seed(8)
tf.random.set_seed(8)

In [ ]:
# Load training data
remove_drugs = True

if remove_drugs:
  train_path = '/content/drive/MyDrive/Colab Notebooks/Input/singlecell/de_train_small.parquet'
  df_train = pd.read_parquet(train_path)
else:
  train_path = '/content/drive/MyDrive/Colab Notebooks/Input/singlecell/de_train.parquet'
  df_train = pd.read_parquet(train_path)

In [ ]:
# Get dose values

if remove_drugs:
  df_logfc = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Input/singlecell/logFC_small.parquet')
  dose = np.array(df_logfc["dose_uM"])
  dose = dose.reshape((df_train.shape[0], 1))
else:
  df_logfc = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Input/singlecell/logFC.parquet')
  dose = np.array(df_logfc["dose_uM"])
  dose = dose.reshape((df_train.shape[0], 1))

In [ ]:
# Get Morgan fingerprints
df_X = np.zeros([df_train.shape[0], 2048])
for i in range(df_train.shape[0]):
	df_X[i, :] = np.array(AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(df_train["SMILES"][i]), radius=2, nBits=2048))

In [ ]:
# One hot encode the cells
# One hot ordering: [B cells,  Myeloid cells,  NK cells,  T cells CD4+,  T cells CD8+,  T regulatory cells]
one_hot = np.array(pd.get_dummies(df_train["cell_type"]) * 1.0)

In [ ]:
# One hot encode controls
# 1 - control, 0 - otherwise
train_control = np.zeros([df_train.shape[0], 1])
for i, cont in enumerate(df_train["control"]):
  if cont == True:
    train_control[i] = 1.

In [ ]:
# Get log p values for train set

if remove_drugs:
  df_chem = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Input/singlecell/chemical_properties_small.csv')
  # Apply Log10 transform to n_atoms, molecular weight, and molar refractivity
  df_chem["n_atoms"] = df_chem["n_atoms"].map(np.log10)
  df_chem["mol_weight"] = df_chem["mol_weight"].map(np.log10)
  df_chem["MR"] = df_chem["MR"].map(np.log10)
else:
  df_chem = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Input/singlecell/chemical_properties.csv')
  # Apply Log10 transform to n_atoms, molecular weight, and molar refractivity
  df_chem["n_atoms"] = df_chem["n_atoms"].map(np.log10)
  df_chem["mol_weight"] = df_chem["mol_weight"].map(np.log10)
  df_chem["MR"] = df_chem["MR"].map(np.log10)

In [ ]:
# Scale data
scaler = StandardScaler()
scaler.fit(df_chem[["log_P", "MR"]])

StandardScaler()

In [ ]:
# Join to form training matrix
# Toggle to include dose and control
incl_control = False
if incl_control:
  one_hot = np.concatenate((one_hot, scaler.transform(df_chem[["log_P", "MR"]])), axis=1)
  one_hot = np.concatenate((one_hot, train_control), axis=1)
  x_train = np.concatenate((one_hot, df_X), axis=1)
else:
  # Normal features to be used
  one_hot = np.concatenate((one_hot, scaler.transform(df_chem[["log_P", "MR"]])), axis=1)
  x_train = np.concatenate((one_hot, df_X), axis=1)

In [ ]:
x_train.shape

(548, 2056)

In [ ]:
# Make train data and targets
n_genes = 18211
y_train = np.array(df_train.iloc[:, 5:])

In [ ]:
# Get log FC
y_logfc = np.array(df_logfc.iloc[:, 7:])

In [ ]:
# Remove control samples. Takes out rows with cell type and compound exposure used as control
# Removing control also reduced the outliers
remove_control = False
if remove_control:
	control_false = list(df_train.index[df_train["control"] == False])
	# x_train and y_train
	x_train = x_train[control_false, :]
	y_train = y_train[control_false, :]
	# df_train
	df_train = df_train[df_train["control"] == False]

In [ ]:
np.min(y_train)

-127.25861351788677

In [ ]:
np.max(y_train)

159.27488278365632

In [ ]:
x_train.shape, y_train.shape, y_logfc.shape

((548, 2056), (548, 18211), (548, 18211))

In [ ]:
# Gavish Donohoe SVD dimension reduction, returns q, U, S, VT
def gd_svd(data_mat, cutoff="w_B_high"):
  U, S, VT = np.linalg.svd(data_mat, full_matrices=False)
  # Calculate aspect ratio and cutoff
  Beta = data_mat.shape[0] / data_mat.shape[1]
  # Approximate w(B)
  w_B = 0.56 * Beta ** 3 - 0.95 * Beta ** 2 + 1.82 * Beta + 1.43
  med_S = np.median(S)
  if cutoff == "w_B":
    tau = w_B * med_S
  elif cutoff == "w_B_low":
    w_B_low = w_B - 0.02
    tau = w_B_low * med_S
  elif cutoff == "w_B_high":
    w_B_high = w_B + 0.02
    tau = w_B_high * med_S
  # Get optimal modes
  q = np.max(np.where(S > tau))
  U, S, VT = U[:, :(q+1)], np.diag(S[:(q+1)]), VT[:(q+1), :]
  return q, U, S, VT

In [ ]:
# Perform dimension reduction on y_train
run_svd = True
run_autoencoder = False

if run_svd:
  q, U, S, VT = gd_svd(data_mat=y_train, cutoff="w_B_high")

if run_autoencoder:
  encoding_dim = 115
  # Load in AE model
  ae_target = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/AE Models/sign_logpv_autoencoder_linear_embed_600e_115.h5')
  # Encoder
  encoder = keras.Model(inputs=ae_target.input, outputs=ae_target.layers[7].output)
  # Decoder
  decoder = keras.Model(inputs=ae_target.layers[8].input, outputs=ae_target.layers[-1].output)

q

100

In [ ]:
# Get embeddings for y_train
if run_svd:
  # Calculate denoised y_train
  y_train_tilde = U @ S @ VT
  # Get y_embed
  y_embed = U @ S

if run_autoencoder:
  # Get y_embed
  y_embed = encoder.predict(y_train)

y_embed.shape

(548, 101)

In [ ]:
# Perform SVD on logfc
if run_svd:
  k, U_lfc, S_lfc, VT_lfc = gd_svd(data_mat=y_logfc, cutoff="w_B_high")

k

61

In [ ]:
# Get embeddings for log FC
if run_svd:
  # Calculate embeddings as features
  y_embed_logfc = U_lfc @ S_lfc

In [ ]:
np.max(y_embed), np.min(y_embed), np.max(y_embed_logfc), np.min(y_embed_logfc)

(518.4002568778097,
 -1572.9470212129154,
 228.0118583629156,
 -418.21068628665535)

In [ ]:
x_train.shape, y_embed.shape, y_embed_logfc.shape

((548, 2056), (548, 101), (548, 62))

In [ ]:
# Augment features with average drug response and average cell type response to drugs
# P-val responses
# Embedding features
embedding_features = False

# Drug response, cell type response
drug_response = False
cell_type_response = False
# Log FC response
drug_lfc_response = False
cell_type_lfc_response = False

if drug_response:
  scaler_drug = StandardScaler()
  if embedding_features:
    df_smiles_name = df_train.iloc[:, [3]]
    df_smiles_name = pd.concat((df_smiles_name, pd.DataFrame(y_embed)), axis=1) # Use SVD embeddings to average over
  else:
    # Normal dimension features
    df_smiles_name = df_train.iloc[:, [3] + list(range(5, df_train.shape[1]))]
  mean_smiles_name = df_smiles_name.groupby('SMILES').mean().reset_index()
  df_train_alt = df_train.iloc[:, 0:5]
  df_train_alt = df_train_alt.merge(mean_smiles_name, on='SMILES', how='left')
  mean_smiles_train = np.array(df_train_alt.iloc[:, 5:])
  # Fit and transform
  scaler_drug.fit(mean_smiles_train)
  # Concat with x_train
  x_train = np.concatenate((x_train, scaler_drug.transform(mean_smiles_train)), axis=1)

if cell_type_response:
  scaler_cell = StandardScaler()
  if embedding_features:
    df_cell_type = df_train.iloc[:, [0]]
    df_cell_type = pd.concat((df_cell_type, pd.DataFrame(y_embed)), axis=1)
  else:
    # Normal dimension features
    df_cell_type = df_train.iloc[:, [0] + list(range(5, df_train.shape[1]))]
  mean_cell_type = df_cell_type.groupby('cell_type').mean().reset_index()
  df_train_alt = df_train.iloc[:, 0:5]
  df_train_alt = df_train_alt.merge(mean_cell_type, on='cell_type', how='left')
  mean_cell_train = np.array(df_train_alt.iloc[:, 5:])
  # Fit and transform
  scaler_cell.fit(mean_cell_train)
  # Concat with x_train
  x_train = np.concatenate((x_train, scaler_cell.transform(mean_cell_train)), axis=1)

if drug_lfc_response:
  scaler_drug_lfc = StandardScaler()
  if embedding_features:
    df_smiles_name = df_train.iloc[:, [3]]
    df_smiles_name = pd.concat((df_smiles_name, pd.DataFrame(y_embed_logfc)), axis=1)
  else:
    # Normal dimension features
    df_smiles_name = df_train.iloc[:, [3]]
    df_smiles_name = pd.concat((df_smiles_name, df_logfc.iloc[:, 7:]), axis=1)
  mean_smiles_lfc_name = df_smiles_name.groupby('SMILES').mean().reset_index()
  df_train_alt = df_train.iloc[:, 0:5]
  df_train_alt = df_train_alt.merge(mean_smiles_lfc_name, on='SMILES', how='left')
  mean_smiles_lfc_train = np.array(df_train_alt.iloc[:, 5:])
  # Fit and transform
  scaler_drug_lfc.fit(mean_smiles_lfc_train)
  # Concat with x_train
  x_train = np.concatenate((x_train, scaler_drug_lfc.transform(mean_smiles_lfc_train)), axis=1)

if cell_type_lfc_response:
  scaler_cell_lfc = StandardScaler()
  if embedding_features:
    df_cell_type = df_train.iloc[:, [0]]
    df_cell_type = pd.concat((df_cell_type, pd.DataFrame(y_embed_logfc)), axis=1)
  else:
    # Normal dimension features
    df_cell_type = df_train.iloc[:, [0]]
    df_cell_type = pd.concat((df_cell_type, df_logfc.iloc[:, 7:]), axis=1)
  mean_cell_lfc_type = df_cell_type.groupby('cell_type').mean().reset_index()
  df_train_alt = df_train.iloc[:, 0:5]
  df_train_alt = df_train_alt.merge(mean_cell_lfc_type, on='cell_type', how='left')
  mean_cell_lfc_train = np.array(df_train_alt.iloc[:, 5:])
  # Fit and transform
  scaler_cell_lfc.fit(mean_cell_lfc_train)
  # Concat with x_train
  x_train = np.concatenate((x_train, scaler_cell_lfc.transform(mean_cell_lfc_train)), axis=1)

In [ ]:
x_train.shape

(548, 2056)

In [ ]:
# Keep copy of x_train in original row order wrt df_train
x_train_copy = x_train
y_train_copy = y_train
y_embed_copy = y_embed

In [ ]:
x_train.shape, y_train.shape, y_embed.shape

((548, 2056), (548, 18211), (548, 101))

In [ ]:
# Sign regression index list 75% accuracy (from sign regression JTT)
idx_list = [  8,   9,  10,  11,  12,  30,  31,  32,  33,  34,  38,  40,  56,
        72,  73,  74,  75,  76,  86,  87,  90,  91,  94,  95,  96,  97,
       118, 130, 151, 154, 155, 156, 157, 158, 168, 169, 170, 171, 174,
       193, 194, 201, 202, 223, 229, 230, 233, 234, 235, 236, 267, 268,
       271, 272, 297, 309, 313, 315, 316, 317, 327, 336, 377, 381, 382,
       405, 409, 410, 411, 412, 413, 414, 419, 447, 448, 451, 452, 456,
       459, 460, 467, 475, 477, 478, 517, 518, 519, 520, 521, 523, 540,
       541, 556, 557, 558, 559, 579, 602, 603, 607]

In [ ]:
# Sign regression index list -2 STD accuracy (from sign regression JTT after drugs not in test removed)
# Index is from 0 to 545, not 0 to 614. This sign based filtering was done on training set with drugs removed
idx_list = [  0,  10,  11,  20,  23,  28,  29,  30,  32,  33,  44,  48,  52,
        53,  55,  64,  65,  70,  74,  75,  78,  79,  82,  98,  99, 110,
       127, 131, 132, 143, 144, 154, 158, 162, 163, 167, 170, 171, 178,
       182, 183, 186, 190, 191, 192, 199, 202, 203, 212, 232, 233, 236,
       237, 254, 255, 258, 259, 262, 263, 270, 273, 276, 278, 280, 281,
       292, 293, 295, 296, 312, 320, 323, 330, 331, 334, 335, 347, 358,
       359, 362, 364, 368, 369, 372, 376, 388, 396, 398, 399, 400, 405,
       408, 409, 412, 413, 416, 417, 420, 421, 423, 426, 442, 443, 446,
       458, 463, 465, 478, 479, 486, 494, 495, 496, 497, 500, 501, 508,
       509, 512, 513, 514, 515, 524, 536, 537, 539, 540, 541, 544, 545]

In [ ]:
len(idx_list)

130

In [ ]:
# Get samples which had large error in train to upsample from
x_train_upsample = x_train[idx_list, :]
y_train_upsample = y_train[idx_list, :]
y_embed_upsample = y_embed[idx_list, :]

x_train_upsample.shape, y_embed_upsample.shape, y_train_upsample.shape

((130, 2056), (130, 101), (130, 18211))

In [ ]:
# Generate new samples
scaling_factor = 6
new_samples_x = x_train_upsample
new_samples_y_train = y_train_upsample
new_samples_y_embed = y_embed_upsample

for i in range(1, scaling_factor, 1):
  new_samples_x = np.concatenate((new_samples_x, x_train_upsample), axis=0)
  new_samples_y_train = np.concatenate((new_samples_y_train, y_train_upsample), axis=0)
  new_samples_y_embed = np.concatenate((new_samples_y_embed, y_embed_upsample), axis=0)

new_samples_x.shape, new_samples_y_embed.shape, new_samples_y_train.shape

((780, 2056), (780, 101), (780, 18211))

In [ ]:
# Concatenate results to original x_train and y_embed
x_train = np.concatenate((x_train, new_samples_x), axis=0)
y_train = np.concatenate((y_train, new_samples_y_train), axis=0)
y_embed = np.concatenate((y_embed, new_samples_y_embed), axis=0)

In [ ]:
x_train.shape, y_embed.shape, y_train.shape

((1328, 2056), (1328, 101), (1328, 18211))

In [ ]:
# Shuffle data
permuted_indices = np.random.permutation(np.arange(x_train.shape[0]))
x_train = x_train[permuted_indices, :]
y_train = y_train[permuted_indices, :]
y_embed = y_embed[permuted_indices, :]

In [ ]:
x_train.shape, y_train.shape, y_embed.shape

((1328, 2056), (1328, 18211), (1328, 101))

In [ ]:
# Model
# Architecture selection - 3 heads for contrastive loss, 3 choose 2 = 3 combinations
# Can also try 2 head contrastive model
def create_3_head(VT_project=False):
  # One input, 3 outputs
  input_size = x_train.shape[1]
  inputs_reg = layers.Input((input_size,))
  x_1 = layers.Dense(5128, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros")(inputs_reg)
  x_2 = layers.Dense(5128, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros")(x_1)
  x_3 = layers.Dense(5128, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros")(x_2)
  x_4 = layers.Dense(5128, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros")(x_3)
  x_5 = layers.Dense(5128, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros")(x_4)
  # Head 1
  h_1_1 = layers.Dense(3072, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros")(x_5)
  h_1_2 = layers.Dense(2048, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros")(h_1_1)
  h_1_3 = layers.Dense(1024, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros")(h_1_2)
  if run_svd:
    if VT_project:
      output_embed_1 = layers.Dense((q+1), kernel_initializer="glorot_normal")(h_1_3)
      output_h_1 = layers.Dense(n_genes, use_bias=False, trainable=False, name="output_h_1")(output_embed_1)
    else:
      output_h_1 = layers.Dense((q+1), kernel_initializer="glorot_normal")(h_1_3)
  if run_autoencoder:
    output_h_1 = layers.Dense(encoding_dim, kernel_initializer="glorot_normal")(h_1_3)
  # Head 2, vary architecture slightly
  h_2_1 = layers.Dense(3072, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros")(x_5)
  h_2_2 = layers.Dense(2048, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros")(h_2_1)
  h_2_3 = layers.Dense(1024, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros")(h_2_2)
  if run_svd:
    if VT_project:
      output_embed_2 = layers.Dense((q+1), kernel_initializer="glorot_normal")(h_2_3)
      output_h_2 = layers.Dense(n_genes, use_bias=False, trainable=False, name="output_h_2")(output_embed_2)
    else:
      output_h_2 = layers.Dense((q+1), kernel_initializer="glorot_normal")(h_2_3)
  if run_autoencoder:
    output_h_2 = layers.Dense(encoding_dim, kernel_initializer="glorot_normal")(h_2_3)
  # Head 3, vary architecture slightly
  h_3_1 = layers.Dense(3072, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros")(x_5)
  h_3_2 = layers.Dense(2048, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros")(h_3_1)
  h_3_3 = layers.Dense(1024, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros")(h_3_2)
  if run_svd:
    if VT_project:
      output_embed_3 = layers.Dense((q+1), kernel_initializer="glorot_normal")(h_3_3)
      output_h_3 = layers.Dense(n_genes, use_bias=False, trainable=False, name="output_h_3")(output_embed_3)
    else:
      output_h_3 = layers.Dense((q+1), kernel_initializer="glorot_normal")(h_3_3)
  if run_autoencoder:
    output_h_3 = layers.Dense(encoding_dim, kernel_initializer="glorot_normal")(h_3_3)
  # Concat layer
  concat = layers.Concatenate(axis=-1)([output_h_1, output_h_2, output_h_3])
  model = keras.Model(inputs=inputs_reg, outputs=concat)
  return model

def create_n_head(n_heads, VT_project=False):
  input_size = x_train.shape[1]
  inputs_reg = layers.Input((input_size,))
  # Shared model weights
  x_1 = layers.Dense(5128, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros", name="shared_1")(inputs_reg)
  x_2 = layers.Dense(5128, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros", name="shared_2")(x_1)
  x_3 = layers.Dense(5128, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros", name="shared_3")(x_2)
  x_4 = layers.Dense(5128, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros", name="shared_4")(x_3)
  x_5 = layers.Dense(5128, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros", name="shared_5")(x_4)
  # Create n_heads - specialise based on loss function used
  concat_list = []
  for i in range(1, n_heads+1, 1):
    globals()['h_' + str(i) + '_1'] = layers.Dense(3072, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros", name="head_" + str(i) + "_l_1")(x_5)
    globals()['h_' + str(i) + '_2'] = layers.Dense(2048, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros", name="head_" + str(i) + "_l_2")(globals()['h_' + str(i) + '_1'])
    globals()['h_' + str(i) + '_3'] = layers.Dense(1024, activation="selu", kernel_initializer="lecun_normal", bias_initializer="zeros", name="head_" + str(i) + "_l_3")(globals()['h_' + str(i) + '_2'])
    if run_svd:
      if VT_project:
        globals()['output_' + 'embed_' + str(i)] = layers.Dense((q+1), kernel_initializer="glorot_normal", name="embed_" + str(i))(globals()['h_' + str(i) + '_3'])
        globals()['output_' + 'h_' + str(i)] = layers.Dense(n_genes, use_bias=False, trainable=False, name="head_" + str(i) + "_out")(globals()['output_' + 'embed_' + str(i)])
      else:
        globals()['output_' + 'h_' + str(i)] = layers.Dense((q+1), kernel_initializer="glorot_normal", name="head_" + str(i) + "_out")(globals()['h_' + str(i) + '_3'])
    if run_autoencoder:
      globals()['output_' + 'h_' + str(i)] = layers.Dense(encoding_dim, kernel_initializer="glorot_normal", name="head_" + str(i) + "_out")(globals()['h_' + str(i) + '_3'])
    # Add output layer to concat list to feed into concat layer
    concat_list.append(globals()['output_' + 'h_' + str(i)])
  # Concat layer
  concat = layers.Concatenate(axis=-1, name="concat_layer")(concat_list)
  model = keras.Model(inputs=inputs_reg, outputs=concat)
  return model

In [ ]:
# Create model
# Select VT projection, if False regression heads are modelled on y_embed
VT_project = False

model = create_3_head(VT_project=VT_project)

# 2/3 head model
#model = create_n_head(n_heads=3, VT_project=VT_project)

In [ ]:
# VT project
# Set last weight matrix as VT projection matrix from SVD
if VT_project:
  w_VT = [VT]
  # Set VT matrix as weights for layers -2, -3, -4
  model.layers[-2].set_weights(w_VT)
  model.layers[-3].set_weights(w_VT)
  model.layers[-4].set_weights(w_VT)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 2056)]               0         []                            
                                                                                                  
 dense (Dense)               (None, 5128)                 1054829   ['input_1[0][0]']             
                                                          6                                       
                                                                                                  
 dense_1 (Dense)             (None, 5128)                 2630151   ['dense[0][0]']               
                                                          2                                       
                                                                                              

In [ ]:
# Define losses
def contrastive_loss_3h(y_true, y_pred):
  # n heads
  n = 3
  # Weights
  w_contrast = 0.5
  # MAE used for regression losses for each head
  loss_fn_1 = keras.losses.MeanAbsoluteError()
  loss_fn_2 = keras.losses.MeanAbsoluteError()
  loss_fn_3 = keras.losses.MeanAbsoluteError()
  # Split concat into chunks
  chunks = tf.split(y_pred, num_or_size_splits=n, axis=1)
  # Calculate pairwise loss for each pair
  c_l = 0.
  for pair in itertools.combinations(chunks, 2):
    c_l += rw_cosine_dissimilarity(pair[0], pair[1])
  # Calculate head regression losses
  loss_list = [loss_fn_1, loss_fn_2, loss_fn_3]
  h_l = 0.
  for i in range(n):
    h_l += loss_list[i](y_true, chunks[i])
  # MAE plus weighted average pairwise cosine dis-similarity
  return h_l + w_contrast * (c_l / 3.)

# Contrastive loss multi losses
def contrastive_loss_multi_3h(y_true, y_pred):
  # n heads
  n = 3
  # Weights - may need to reduce w_contrast as some losses are smaller in value numerically
  w_contrast = 0.5 # 0.5 value corresponds to a 36% decrease in contrast loss commensurate with smaller loss sum from use of logcosh and huber
  # Define regression losses for each head
  # MAE, logcosh, huber losses
  mae = keras.losses.MeanAbsoluteError()
  logcosh = keras.losses.LogCosh()
  huber = keras.losses.Huber()
  # Split concat into cunks
  chunks = tf.split(y_pred, num_or_size_splits=3, axis=1)
  # Calculate pairwise loss for each pair
  c_l = 0.
  for pair in itertools.combinations(chunks, 2):
    c_l += rw_cosine_dissimilarity(pair[0], pair[1])
  # Calculate head regression losses
  loss_list = [mae, logcosh, huber]
  h_l = 0.
  for i in range(n):
    h_l += loss_list[i](y_true, chunks[i])
  # Return total losses
  return h_l + w_contrast * (c_l / 3)

# Multi-head additive loss (can add contratistive)
# Can try cosine similarity
def multi_loss(y_true, y_pred):
  # n heads
  n = 8
  # Split into chunks e.g.
  chunks = tf.split(y_pred, num_or_size_splits=8, axis=1)
  # Define losses
  mae = keras.losses.MeanAbsoluteError()
  logcosh = keras.losses.LogCosh()
  huber = keras.losses.Huber() # rwrmse, pseudo_huber, alpha_1point75, alpha_1point5, alpha_1point25, alpha_point5, cauchy, geman_mcclure
  # cosine = keras.losses.CosineSimilarity()
  # Define loss list for each head
  loss_list = [mae, logcosh, huber, pseudo_huber, alpha_1point75, alpha_1point5, alpha_1point25, alpha_1point125]
  h_l = 0.
  for i in range(n):
    h_l += loss_list[i](y_true, chunks[i])
  # Return total losses
  return h_l

# Multi-head additive loss with contrast
def multi_loss_contrastive(y_true, y_pred):
  # n heads
  n = 8
  # Weights
  w_contrast = 0.8
  # Split into chunks e.g.
  chunks = tf.split(y_pred, num_or_size_splits=8, axis=1)
  # Define losses
  mae = keras.losses.MeanAbsoluteError()
  logcosh = keras.losses.LogCosh()
  huber = keras.losses.Huber() # rwrmse, pseudo_huber, alpha_1point75, alpha_1point5, alpha_1point25, alpha_point5, cauchy, geman_mcclure
  # cosine = keras.losses.CosineSimilarity()
  # Calculate pairwise dis-similarity
  c_l = 0.
  for pair in itertools.combinations(chunks, 2):
    c_l += rw_cosine_dissimilarity(pair[0], pair[1])
  # Define loss list for each head
  loss_list = [mae, logcosh, huber, pseudo_huber, alpha_1point75, alpha_1point5, alpha_1point25, alpha_1point125]
  h_l = 0.
  for i in range(n):
    h_l += loss_list[i](y_true, chunks[i])
  # Return total losses
  return h_l + w_contrast * (c_l / 28)

In [ ]:
# Average chunk rwrmse loss
# Change n to 3 if using contrastive loss, else change to number of heads for ensemble
def multi_rwrmse(y_true, y_pred):
	# n heads
	n = 3
	# Combine y_true
	y_true_list = []
	for i in range(n):
		y_true_list.append(y_true)
	y_true_comb = tf.concat(y_true_list, axis=1)
	return rwrmse(y_true_comb, y_pred)

In [ ]:
# Stochastic weight averaging
# Try SWA averaging starting at epoch 1,000
start_epoch = 2
swa = SWA(start_epoch=start_epoch,
          lr_schedule='manual',
          verbose=1)

# Optimizer
# Best scores used learning rate 7e-5, and SWA learning rate 5e-5
learning_rate = 7e-5 # 7e-5 for AdamW, 1e-5 / 7e-6 using Lion

# Cosine rate scheduler
cos_sched = keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=learning_rate, first_decay_steps=16600, t_mul=1.0, m_mul=0.9, alpha=0.01)

# Optimisers
# Try higher weight decay for larger models
# lambda = lambda(norm) * sqrt(batch_size / (n * t)), where n is number of training points and t is number of epochs
# Try lambda(norm) between 0.025 to 0.05
weight_decay = 0.1
opt = keras.optimizers.legacy.Adam(learning_rate=cos_sched)

opt_adamW = keras.optimizers.AdamW(learning_rate=cos_sched)

opt_lion = keras.optimizers.Lion(learning_rate=cos_sched, weight_decay=weight_decay)

In [ ]:
# Comiple model
# Contrastive loss - contrastive_loss all 3 heads evaluated on MAE
model.compile(loss=contrastive_loss_3h, optimizer=opt_adamW, metrics=[multi_rwrmse])

# Train model
model.fit(x=x_train, y=y_embed, epochs=800, batch_size=16, callbacks=[swa], shuffle=True)

Epoch 1/800
 5/83 [>.............................] - ETA: 2s - loss: 14.9031 - multi_rwrmse: 8.8050

83/83 [==============================] - 21s 28ms/step - loss: 13.0214 - multi_rwrmse: 7.7852

Epoch 00002: starting stochastic weight averaging
Epoch 2/800
83/83 [==============================] - 4s 54ms/step - loss: 10.6378 - multi_rwrmse: 6.7766
Epoch 3/800
83/83 [==============================] - 3s 40ms/step - loss: 9.4438 - multi_rwrmse: 6.0894
Epoch 4/800
83/83 [==============================] - 3s 40ms/step - loss: 8.6250 - multi_rwrmse: 5.6540
Epoch 5/800
83/83 [==============================] - 3s 40ms/step - loss: 8.1695 - multi_rwrmse: 5.3780
Epoch 6/800
83/83 [==============================] - 4s 43ms/step - loss: 7.8064 - multi_rwrmse: 5.1525
Epoch 7/800
83/83 [==============================] - 5s 55ms/step - loss: 7.3809 - multi_rwrmse: 4.8915
Epoch 8/800
83/83 [==============================] - 3s 38ms/step - loss: 6.9958 - multi_rwrmse: 4.6258
Epoch 9/800
83/83 [==============================] - 3s 39ms/step - loss: 6.8120 - multi_rwrmse: 4.4703
Epoch 10/800
83/83 [==

In [ ]:
incl_control

False

In [ ]:
VT.shape

(101, 18211)

In [ ]:
# Load in submission set
df_id = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Input/singlecell/id_map_submission.csv')

# Convert to Morgan fingerprints
df_X = np.zeros([df_id.shape[0], 2048])
for i in range(df_id.shape[0]):
	df_X[i, :] = np.array(AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(df_id["SMILES"][i]), radius=2, nBits=2048))

# One hot encode the cell types
n_cell_types = 6
cells = ["B cells",  "Myeloid cells", "NK cells", "T cells CD4+", "T cells CD8+", "T regulatory cells"]
one_hot = pd.DataFrame(np.zeros([df_id.shape[0], n_cell_types]), columns=cells)

# Fill in one_hot
one_hot_test = pd.get_dummies(df_id["cell_type"]) * 1.0
one_hot["B cells"] = one_hot_test["B cells"]
one_hot["Myeloid cells"] = one_hot_test["Myeloid cells"]
one_hot = np.array(one_hot)

# Control and dose for test
# Control (1 - control, 0 - non-control)
test_control = np.zeros([df_id.shape[0], 1])
# Dose (1 uM for all compounds)
test_dose = np.ones((df_id.shape[0], 1))

# Log10 transform n_atoms, molecular weight, and molar refractivity
df_id["n_atoms"] = df_id["n_atoms"].map(np.log10)
df_id["mol_weight"] = df_id["mol_weight"].map(np.log10)
df_id["MR"] = df_id["MR"].map(np.log10)

# Merge to construct test matrix
if incl_control:
	# Test matrix if controls and dose used
	one_hot = np.concatenate((one_hot, scaler.transform(df_id[["log_P", "MR"]])), axis=1)
	one_hot = np.concatenate((one_hot, test_control), axis=1)
	x_test = np.concatenate((one_hot, df_X), axis=1)
else:
	# Test matrix if controls / dose not used
	one_hot = np.concatenate((one_hot, scaler.transform(df_id[["log_P", "MR"]])), axis=1)
	x_test = np.concatenate((one_hot, df_X), axis=1)

del one_hot, one_hot_test

In [ ]:
# Augment features with average drug response and average cell type response to drugs for test

if drug_response:
  df_id_alt = df_id.iloc[:, 0:5]
  df_id_alt = df_id_alt.merge(mean_smiles_name, on='SMILES', how='left')
  mean_smiles_test = np.array(df_id_alt.iloc[:, 5:])
  # Concat with x_test
  x_test = np.concatenate((x_test, scaler_drug.transform(mean_smiles_test)), axis=1)

if cell_type_response:
  df_id_alt = df_id.iloc[:, 0:5]
  df_id_alt = df_id_alt.merge(mean_cell_type, on='cell_type', how='left')
  mean_cell_test = np.array(df_id_alt.iloc[:, 5:])
  # Concat with x_Test
  x_test = np.concatenate((x_test, scaler_cell.transform(mean_cell_test)), axis=1)

if drug_lfc_response:
  df_id_alt = df_id.iloc[:, 0:5]
  df_id_alt = df_id_alt.merge(mean_smiles_lfc_name, on='SMILES', how='left')
  mean_smiles_lfc_test = np.array(df_id_alt.iloc[:, 5:])
  # Concat with x_test
  x_test = np.concatenate((x_test, scaler_drug_lfc.transform(mean_smiles_lfc_test)), axis=1)

if cell_type_lfc_response:
  df_id_alt = df_id.iloc[:, 0:5]
  df_id_alt = df_id_alt.merge(mean_cell_lfc_type, on='cell_type', how='left')
  mean_cell_lfc_test = np.array(df_id_alt.iloc[:, 5:])
  # Concat with x_test
  x_test = np.concatenate((x_test, scaler_cell_lfc.transform(mean_cell_lfc_test)), axis=1)

In [ ]:
x_test.shape

(255, 2056)

In [ ]:
# Predict on x_train copy and split into chunks
nh = 3
output_emb = np.split(model.predict(x_train_copy), nh, axis=1)

# Choose output type based on VT project
if VT_project:
  # Ensemble weights
  weights = []
  for i in range(nh):
    weights.append(K.eval(rwrmse(y_train_copy, output_emb[i])))

  # Construct ensembling weights
  weights = np.array(weights)
  weights = 1. - weights
  weights = weights / np.sum(weights)

  # Print losses
  y_output_loss = []
  for i in range(nh):
    y_output_loss.append(K.eval(rwrmse(y_train_copy, output_emb[i])))
    print(i, y_output_loss[i])
else:
  # Ensemble weights
  weights = []
  for i in range(nh):
    weights.append(K.eval(rwrmse(y_train_copy, (output_emb[i] @ VT))))

  # Construct ensembling weights
  weights = np.array(weights)
  weights = 1. - weights
  weights = weights / np.sum(weights)

  # Embedding high
  y_output_loss = []
  for i in range(nh):
    y_output_loss.append(K.eval(rwrmse(y_train_copy, (output_emb[i] @ VT))))
    print(i, y_output_loss[i])

18/18 [==============================] - 0s 3ms/step
0 0.5125406851086056
1 0.5112158307464528
2 0.5119393549501484


In [ ]:
# Select the best head and also loss weight ensemble the 3 heads
# Head index
head_idx = y_output_loss.index(min(y_output_loss))

In [ ]:
head_idx

1

In [ ]:
# Predict on test and split into chunks
# Best head prediction (head_idx)
output_emb = np.split(model.predict(x_test), nh, axis=1)

if VT_project:
  output_high = output_emb[head_idx]
else:
  if run_svd:
    output_high = output_emb[head_idx] @ VT
  if run_autoencoder:
    output_high = decoder.predict(output_emb[head_idx])

# Read in submission file
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Input/singlecell/sample_submission.csv')
submission.iloc[:, 1:] = output_high

# Save as submission
# Generate truncated SHA256 hash
truncated_hash = hashlib.sha256(os.urandom(23)).hexdigest()[:23]

submission_path = '/content/drive/MyDrive/Colab Notebooks/Output/ecfp_reg_8lselu_multi_v485_' + truncated_hash + '.csv'
submission.to_csv(submission_path, index=False)

# Print hash
print(truncated_hash)

8/8 [==============================] - 0s 4ms/step
082f032913803e988aff81d


In [ ]:
np.max(output_high), np.min(output_high)

(75.41930454532292, -27.907574788717152)

In [ ]:
# Use equal weights
weights_equal = False

if weights_equal:
  weights = [1/nh] * nh

In [ ]:
weights

array([0.33289486, 0.33379963, 0.33330552])

In [ ]:
# Get ensemble (multi-head embedding blend)
# Ensemble of embedding outputs and inverse transform with VT
output_high = np.zeros([255, n_genes])

for i in range(nh):
  if VT_project:
    # If using SVD VT project matrix
    output_high += (weights[i] * output_emb[i])
  else:
    output_high += (weights[i] * (output_emb[i] @ VT))

# Read in submission file
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Input/singlecell/sample_submission.csv')
submission.iloc[:, 1:] = output_high

# Generate truncated SHA256 hash
truncated_hash = hashlib.sha256(os.urandom(23)).hexdigest()[:23]

submission_path = '/content/drive/MyDrive/Colab Notebooks/Output/ecfp_reg_8lselu_multi_ensemble_low_v486_' + truncated_hash + '.csv'
submission.to_csv(submission_path, index=False)

# Print hash
print(truncated_hash)

191ef44d952ae115a60171e


In [ ]:
np.max(output_high), np.min(output_high)

(74.01194911064934, -28.12256482084174)

In [ ]:
# Save model
# Use .keras extension to save whole model
filepath = '/content/drive/MyDrive/Colab Notebooks/Models/ecfp_reg_8lselu_stack_v486_' + truncated_hash + '.keras'
model.save(filepath)